# Inżynieria cech - data split



In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [19]:
data = pd.read_csv("Titanic2.txt", index_col=0)
display(data.head())

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,boat_null,cabin_null,body_null,home.dest_null,CabinReduced
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO",0,0,1,0,B
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON",0,0,1,0,C
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON",1,0,1,0,C
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON",1,0,0,0,C
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON",1,0,1,0,C


## Funkcja train_test_split
Funkcja **train_test_split** z pakietu Scikit-learn pozwala na podział danych na zbiór testowy i treningowy, co pomaga w wytrenowaniu modelu i w obiektywnej jego ocenie.

Do funkcji podajemy *nasze dane* w formie jednej lub więcej macierzy, możemy zdefiniować *test_size*, czyli rozmiar zestawu testowego (lub zamiennie możemy ustawić train_size), *random_state*, czyli parametr umożliwiający uzyskanie powtarzalności podziału, parametr *shuffle*, który ustala, czy dane powinny zostać przemieszane (potasowane), a jako ostatnie za pomocą parametru *stratify* możemy ustawić zestaw etykiet, które powinny być równomiernie zachowane w każdym ze zbiorów.

Funkcja w fajny sposób pozwala na sterowanie parametrami podziału.

## Tworzenie podziału

In [20]:
col_name = ["cabin", "CabinReduced", "sex"]

X = data[col_name]
y = data["survived"]

# Podział na zestawy treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print(data.shape)
print(f"Wymiary: X_train {X_train.shape}, X_test {X_test.shape}, y_train {y_train.shape}, y_test {y_test.shape}")

(1309, 19)
Wymiary: X_train (1047, 3), X_test (262, 3), y_train (1047,), y_test (262,)


Można zauważyć że pierwszy z wymiarów danych treningowych to 80% liczby wszystkich danych, a dla danych testowych jest to 20% liczebności, co zgadzałoby się z ustawionym parametrem **"test_size"**.

Brak drugiego wymiaru dla danych *y* wskazuje, że jest to jedna kolumna. Z kolei dla danych *X* drugi wymiar jest równy liczbie cech, które zostały wybrane.

Proporcje danych treningowych i testowych są zgodne z tym, czego oczekiwaliśmy w naszym podziale danych.

## Badanie rozkładu etykiet
Obliczam różnicę między unikalnymi wartościami cech w zbiorach treningowym i testowym, aby zbadać, czy rozkład etykiet w tych zbiorach jest równomierny

In [21]:
unique_test_cabin = [x for x in X_test.cabin.unique() if x not in X_train.cabin.unique()]
len(unique_test_cabin)

24

In [22]:
unique_test_CabinReduced = [x for x in X_test.CabinReduced.unique() if x not in X_train.CabinReduced.unique()]
len(unique_test_CabinReduced)

1

In [23]:
unique_test_sex = [x for x in X_test.sex.unique() if x not in X_train.sex.unique()]
len(unique_test_sex)

0

Dla zmiennej cabin aż 24 unikalne wartości ze zbioru testowego nie występują w zbiorze treningowym, co nie jest dobrą sytuacją. Dla zmiennej CabinReduced jest to 1 wartość, a dla płci 0. Tylko dla ostatniej cechy rozkład został całkowicie zachowany.

Postanowiłam sprawdzić czym jest ta jedna wartość w przypadku CabinReduced.

In [24]:
print(data.CabinReduced.unique())
print(unique_test_CabinReduced)

['B' 'C' 'E' 'D' 'A' nan 'T' 'F' 'G']
[nan]


## Kodowanie zmiennych kategorycznych

In [25]:
dictionaries = []
for col in col_name:
  dicki = {item:count for count, item in enumerate(data[col].unique())}
  dictionaries.append(dicki)


for i, col in enumerate(col_name):
  X_test[col+'_map'] = X_test[col].map(dictionaries[i])
  X_train[col+'_map'] = X_train[col].map(dictionaries[i])

display(X_test.head(10))

,cabin,CabinReduced,sex,cabin_map,CabinReduced_map,sex_map
1139,NaN,NaN,male,6,5,1
533,NaN,NaN,female,6,5,0
459,NaN,NaN,male,6,5,1
1150,NaN,NaN,male,6,5,1
393,NaN,NaN,male,6,5,1
1189,G6,G,female,185,8,0
5,E12,E,male,2,2,1
231,C104,C,male,130,1,1
330,NaN,NaN,male,6,5,1
887,NaN,NaN,male,6,5,1


## Liczba brakujących wartości w nowych kolumnach

In [26]:
for col in col_name:
  print(f'Liczba brakujących wartości dla {col.upper()+"_map"} w X_train: {X_train[col+"_map"].isnull().sum()}, oraz w X_test: {X_train[col+"_map"].isnull().sum()}')

Liczba brakujących wartości dla CABIN_map w X_train: 0, oraz w X_test: 0
Liczba brakujących wartości dla CABINREDUCED_map w X_train: 0, oraz w X_test: 0
Liczba brakujących wartości dla SEX_map w X_train: 0, oraz w X_test: 0


Po zmapowaniu kolumn liczba brakujących wartości we wszystkich nowych kolumnach wynosi 0. Wynika to z tego, że wartości NaN została przydzielona konkretna etykieta liczbowa.Każda unikalna etykieta została przypisana do odpowiedniego kodu liczbowego - NaN nie stanowi tu wyjątku.

Warto dodać, że jest tak zarówno dla danych testowych jak i treningowych, dlatego że mapowania zostały wykonane osobno dla obu zestawów.

## Zastąpienie NaN zerami

In [27]:
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)
display(X_test.head(10))

,cabin,CabinReduced,sex,cabin_map,CabinReduced_map,sex_map
1139,0,0,male,6,5,1
533,0,0,female,6,5,0
459,0,0,male,6,5,1
1150,0,0,male,6,5,1
393,0,0,male,6,5,1
1189,G6,G,female,185,8,0
5,E12,E,male,2,2,1
231,C104,C,male,130,1,1
330,0,0,male,6,5,1
887,0,0,male,6,5,1


Zastępowanie brakujących wartości zerami wydaje się być rozsądne, ale głównie dla zmiennych numerycznych, czyli gdy to 0 ma szanse w ogóle występować.

W przypadku kolumn opisujących kabinę pasażera metoda ta może nie być zbyt stosowna.

## Liczba unikalnych wartości po redukcji, po zmapowaniu

#### Unikalne wartości w obrębie kolumn

In [28]:
unique_train_values = [len(X_train[col].unique()) for col in X_train.columns]
unique_test_values = [len(X_test[col].unique()) for col in X_test.columns]

print(f'Nazwy kolumn w tablicach: {X_train.columns}')
print(f'Liczba unikalnych wartości dla danych treningowych {unique_train_values}')
print(f'Liczba unikalnych wartości dla danych testowych {unique_test_values}')


Nazwy kolumn w tablicach: Index(['cabin', 'CabinReduced', 'sex', 'cabin_map', 'CabinReduced_map',
       'sex_map'],
      dtype='object')
Liczba unikalnych wartości dla danych treningowych [164, 9, 2, 164, 9, 2]
Liczba unikalnych wartości dla danych testowych [49, 8, 2, 49, 8, 2]


#### Rozkład etykiet

In [30]:
unique_test_cabin = [x for x in X_test.cabin_map.unique() if x not in X_train.cabin_map.unique()]
print(len(unique_test_cabin))

unique_test_CabinReduced = [x for x in X_test.CabinReduced_map.unique() if x not in X_train.CabinReduced_map.unique()]
print(len(unique_test_CabinReduced))

unique_test_sex = [x for x in X_test.sex.unique() if x not in X_train.sex.unique()]
len(unique_test_sex)

23
0


0

## Wnioski
Liczba unikalnych wartości w obrębie kolumn nie zmieniła się po procesie mapowania. Dodatkowo, liczba unikalnych wartości ze zbioru testowego, które nie występują w zbiorze treningowym zmniejszyła sie dla cabin oraz CabinReduced o 1.

Cały wykonany proces przetwarzania danych może mieć wpływ na dopasowanie modelu uczenia maszynowego.